<a id="contents"></a>


# Contents 

* [Install packages on all machines via dask](#install)   
* [Image preprocessing via dask](#pre)
* [sklearn : SGD](#sgd)
* [sklearn dask integration: Grid search](#grid)
* [sklearn dask integration: SVC](#svc)

In [38]:
from dask.distributed import Client
client = Client('dask-scheduler:8786')
client

<Client: 'tcp://172.18.0.2:8786' processes=6 threads=6, memory=12.85 GiB>

<a href ="#contents">
<p style="text-align: right;">return to contents</p>
</a>  
<a id="install"></a> 


# Install skimage and dask image

In [2]:
import os
os.system('conda install -c anaconda scikit-image -y')

def skimage_install():
    out=os.system('conda install -c anaconda scikit-image -y')
    return out

client.run(skimage_install)

Solving environment: ...working... done




==> WARNING: A newer version of conda exists. <==
  current version: 4.12.0
  latest version: 4.13.0

Please update conda by running

    $ conda update -n base -c defaults conda





## Package Plan ##

  environment location: /opt/conda

  added / updated specs:
    - scikit-image


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    blosc-1.21.0               |       h8c45485_0          64 KB
    brunsli-0.1                |       h2531618_0         199 KB  anaconda
    ca-certificates-2022.4.26  |       h06a4308_0         132 KB  anaconda
    certifi-2022.6.15          |   py39h06a4308_0         156 KB  anaconda
    cfitsio-3.470              |       hf0d0db6_6         1.3 MB  anaconda
    charls-2.2.0               |       h2531618_0         129 KB  anaconda
    conda-4.13.0               |   py39h06a4308_0         995 KB  anaconda
    imagecodecs-2021.8.26      |   py39h4cda21f_0         7.0 MB  anaconda
    imageio-2.9.0              |     pyhd3eb1b0_0         3.1 MB  anaconda
    jxrlib-1.1                 |       h7b6447c_2         238 KB  anaconda
    lerc-3.0 

{'tcp://172.18.0.4:37083': 0,
 'tcp://172.18.0.5:39467': 0,
 'tcp://172.18.0.6:37063': 0,
 'tcp://172.18.0.7:46687': 0,
 'tcp://172.18.0.8:43201': 0,
 'tcp://172.18.0.9:38355': 0}

In [3]:
os.system('conda install -c conda-forge dask-image -y')

def skimage_install():
    out=os.system('conda install -c conda-forge dask-image -y')
    return out

client.run(skimage_install)

Solving environment: ...working... done

## Package Plan ##

  environment location: /opt/conda

  added / updated specs:
    - dask-image


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2022.6.15  |       ha878542_0         149 KB  conda-forge
    certifi-2022.6.15          |   py39hf3d152e_0         155 KB  conda-forge
    conda-4.13.0               |   py39hf3d152e_1         998 KB  conda-forge
    dask-image-2021.12.0       |     pyhd8ed1ab_0          47 KB  conda-forge
    pims-0.6.1                 |     pyhd8ed1ab_0          66 KB  conda-forge
    python_abi-3.9             |           2_cp39           4 KB  conda-forge
    slicerator-1.1.0           |     pyhd8ed1ab_0          13 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         1.4 MB

The following NEW packages will be INS

{'tcp://172.18.0.4:37083': 0,
 'tcp://172.18.0.5:39467': 0,
 'tcp://172.18.0.6:37063': 0,
 'tcp://172.18.0.7:46687': 0,
 'tcp://172.18.0.8:43201': 0,
 'tcp://172.18.0.9:38355': 0}

<a href ="#contents">
<p style="text-align: right;">return to contents</p>
</a>  
<a id="pre"></a> 

## Dask preprocessing

In [39]:
import numpy as np
import dask_image
import dask.array as da
import dask.dataframe as dd
import dask.delayed
from dask.array.image import imread
filename_pattern = os.path.join('temp-tiles', 'image-*.png')

def load_turtles(path, kind='train'):
    f  = os.path.join(path, 'face/face*.jpg')
    nf = os.path.join(path, 'noface/noface*.jpg')

    #load all images
    images = da.concatenate([imread(f), imread(nf)])
        
    ##create numpy list of ,matching labels
    face_lb = np.ones(len(os.listdir(os.path.join(path, 'face')))) #fetch the 
    no_face_lb= np.negative(np.ones(len(os.listdir(os.path.join(path, 'noface')))))
    labels = np.append(face_lb, no_face_lb)
    labels= da.from_array(labels)
    return images, labels 

data_path = '/opt/workspace/data/'
X, y = load_turtles(data_path)
y

dask.array<array, shape=(4000,), dtype=float64, chunksize=(4000,), chunktype=numpy.ndarray>

## Just testing doing the preprocessing straight away before splitting

We actually are reducing the dimensionality of the array as we convert the images into 2D from RGB array. 

In [40]:
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import cross_val_predict
from sklearn.preprocessing import StandardScaler, Normalizer
from skimage.feature import hog
import skimage

class RGB2GrayTransformer(BaseEstimator, TransformerMixin):
    """
    Convert an array of RGB images to grayscale
    """
 
    def __init__(self):
        pass
 
    def fit(self, X, y=None):
        """returns itself"""
        return self
 
    def transform(self, X, y=None):
        """perform the transformation and return an array"""
        return np.array([skimage.color.rgb2gray(img) for img in X])
     
class HogTransformer(BaseEstimator, TransformerMixin):
    """
    Expects an array of 2d arrays (1 channel images)
    Calculates hog features for each img
    """
 
    def __init__(self, y=None, orientations=9,
                 pixels_per_cell=(8, 8),
                 cells_per_block=(3, 3), block_norm='L2-Hys'):
        self.y = y
        self.orientations = orientations
        self.pixels_per_cell = pixels_per_cell
        self.cells_per_block = cells_per_block
        self.block_norm = block_norm
 
    def fit(self, X, y=None):
        return self
 
    def transform(self, X, y=None):
 
        def local_hog(X):
            return hog(X,
                       orientations=self.orientations,
                       pixels_per_cell=self.pixels_per_cell,
                       cells_per_block=self.cells_per_block,
                       block_norm=self.block_norm)
 
        try: # parallel
            return np.array([local_hog(img) for img in X])
        except:

            return np.array([local_hog(img) for img in X])

In [41]:
@dask.delayed
def SGD_preprocessing_d(X):
    # create an instance of each transformer
    grayify = RGB2GrayTransformer()
    hogify = HogTransformer(
        pixels_per_cell=(14, 14), 
        cells_per_block=(2,2), 
        orientations=9, 
        block_norm='L2-Hys'
    )
    scalify = StandardScaler()
    # call fit_transform on each transform converting X_train step by step
    X_gray = grayify.fit_transform(X)
    X_hog = hogify.fit_transform(X_gray)
    X_prepared = scalify.fit_transform(X_hog)
    #return np.shape(X_prepared)
    return X_prepared

chunk_size = 500
X=X.rechunk(chunks= (chunk_size, 198,198, 3)).to_delayed().ravel()

#X_prepared = SGD_preprocessing_d(X)
#dd.compute(*[SGD_preprocessing_d(i) for i in X])
#dd.compute(*[da.from_delayed(SGD_preprocessing_d(i), (1,6084), dtype=np.float64) for i in X])
X=da.concatenate([da.from_delayed(SGD_preprocessing_d(i), (chunk_size, 6084), dtype=np.float64) for i in X])

In [52]:
from IPython.display import display
permutation = da.random.permutation(len(X))

X = X[permutation]
y = y[permutation]

m_train = int(len(X)*0.6) 
m_test = len(X) - m_train

X_train, X_test = X[:m_train], X[m_train:]
y_train, y_test = y[:m_train], y[m_train:]
print(m_train, m_test, m_train+m_test)

labels, freqs = np.unique(y_train, return_counts=True)
print("Labels in training dataset: ", labels)
print("Frequencies in training dataset: ", freqs)
display(X_train, y_train)
display(X_test, y_test)

2400 1600 4000
Labels in training dataset:  dask.array<getitem, shape=(nan,), dtype=float64, chunksize=(nan,), chunktype=numpy.ndarray>
Frequencies in training dataset:  dask.array<getitem, shape=(nan,), dtype=int64, chunksize=(nan,), chunktype=numpy.ndarray>


dask.array<getitem, shape=(2400, 6084), dtype=float64, chunksize=(2400, 6084), chunktype=numpy.ndarray>

dask.array<getitem, shape=(2400,), dtype=float64, chunksize=(2400,), chunktype=numpy.ndarray>

dask.array<getitem, shape=(1600, 6084), dtype=float64, chunksize=(1600, 6084), chunktype=numpy.ndarray>

dask.array<getitem, shape=(1600,), dtype=float64, chunksize=(1600,), chunktype=numpy.ndarray>

In [55]:
from dask_ml.datasets import make_classification
from dask_ml.wrappers import Incremental
from sklearn.linear_model import SGDClassifier

X_train = X_train.rechunk(chunks= (400, 6084))
y_train = y_train.rechunk(chunks= (400))

##maybe rechunck the test
X_train = X_train.compute()
y_train = y_train.compute()
X_test = X_test.compute()
y_test = y_test.compute()

<a href ="#contents">
<p style="text-align: right;">return to contents</p>
</a>  
<a id="grid"></a> 

## Sklearn: Grid search with dask

In [25]:
import joblib
from sklearn.datasets import load_digits
from sklearn.model_selection import RandomizedSearchCV
from sklearn.svm import SVC

model = SVC(kernel='rbf')
search = RandomizedSearchCV(model, param_space, cv=3, n_iter=50, verbose=10)

sgd_clf = SGDClassifier(random_state=42, max_iter=1000, tol=1e-3, n_jobs =6)

with joblib.parallel_backend('dask'):
    #     search.fit(digits.data, digits.target)
    sgd_clf.fit(X_train, y_train)

In [26]:
import numpy as np
from dask.distributed import Client
import joblib
from sklearn.model_selection import RandomizedSearchCV
from sklearn.svm import SVC

param_space = {
    'C': np.logspace(-6, 6, 13),
    'gamma': np.logspace(-8, 8, 17),
    'tol': np.logspace(-4, -1, 4),
    'class_weight': [None, 'balanced'],
}

model = SVC(kernel='rbf')
search = RandomizedSearchCV(model, param_space, cv=3, n_iter=50, verbose=10)

with joblib.parallel_backend('dask'):
    search.fit(X_train, y_train)

Fitting 3 folds for each of 50 candidates, totalling 150 fits


In [32]:
search.best_score_

0.9900000000000001

In [34]:
search.best_params_

{'tol': 0.001, 'gamma': 0.0001, 'class_weight': 'balanced', 'C': 100.0}

<a href ="#contents">
<p style="text-align: right;">return to contents</p>
</a>  
<a id="SVC"></a> 

## Sklearn: SVC dask

In [70]:
from sklearn.pipeline import make_pipeline
SVC_model = SVC(kernel='rbf', tol=0.001, gamma=0.0001, class_weight='balanced', C=100)

with joblib.parallel_backend('dask'):
    SVC_model.fit(X_train, y_train)

In [73]:
SVC_train_score = SVC_model.score(X_train,y_train)
SVC_test_score = SVC_model.score(X_test,y_test)
print ("Best SVC training accuracy: %f" % SVC_train_score)
print ("Best SVC test accuracy: %f" % SVC_test_score)

Best SVC training accuracy: 1.000000
Best SVC test accuracy: 0.997500


In [76]:
y_pred = SVC_model.predict(X_test)
fails=X_test[y_pred!=y_test]
fails

array([[-0.27792659, -0.5679682 ,  1.51138673, ...,  0.8377995 ,
         0.84847436,  1.01277626],
       [ 1.12118581, -0.44625991, -0.08157321, ...,  2.02719113,
         1.26765639,  0.39373655],
       [-1.45433081, -0.80322712, -0.96178526, ...,  1.49718345,
        -0.29748138, -0.94892783],
       [-0.39280102, -0.18861421, -0.73141948, ...,  1.22361009,
         0.36761088, -0.69014881]])